# 문화/게임 콘텐츠 용어 LLM 파인튜닝

## 모델 정보
- **Base Model**: `skt/kogpt2-base-v2`
- **Fine-tuning 방식**: LoRA
- **태스크**: 용어 정의 생성

---

## 1. 라이브러리 설치

In [1]:
%%capture
!pip install transformers datasets accelerate peft bitsandbytes
!pip install sentencepiece
!pip install wandb  # 학습 로깅용 (선택사항)

In [2]:
# GPU 확인
!nvidia-smi

Sat Dec  6 03:56:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. 환경 설정 및 라이브러리 임포트

In [3]:
import torch
import json
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

# GPU 사용 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

사용 디바이스: cuda
GPU: Tesla T4
GPU 메모리: 14.7 GB


In [4]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
DATA_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/processed/')
OUTPUT_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/model/')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"데이터 경로: {DATA_PATH}")
print(f"모델 저장 경로: {OUTPUT_PATH}")

Mounted at /content/drive
데이터 경로: /content/drive/MyDrive/LLM_FineTuning_Project/processed
모델 저장 경로: /content/drive/MyDrive/LLM_FineTuning_Project/model


## 3. 모델 및 토크나이저 로드

In [5]:
# 모델명
MODEL_NAME = "skt/kogpt2-base-v2"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)

print(f"토크나이저 로드 완료!")
print(f"어휘 크기: {len(tokenizer):,}")

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

토크나이저 로드 완료!
어휘 크기: 51,200


In [7]:
# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map='auto',
    trust_remote_code=True
)
model.config.use_cache = False

print(f"모델 로드 완료!")
print(f"모델 파라미터 수: {model.num_parameters():,}")

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

모델 로드 완료!
모델 파라미터 수: 125,164,032


## 4. LoRA 설정

In [8]:
# 모델 구조 확인 (target_modules 찾기용)
print("모델 모듈 이름 확인:")
for name, module in model.named_modules():
    if 'attn' in name.lower() or 'proj' in name.lower():
        print(f"  {name}")

# LoRA 설정 (KoGPT2용)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                        # LoRA rank
    lora_alpha=32,              # LoRA alpha
    lora_dropout=0.1,           # Dropout
    target_modules=["c_attn"],  # KoGPT2의 attention 모듈
    inference_mode=False,       # 학습 모드
    bias="none",                # bias 학습 안함
)

# LoRA를 위한 gradient 활성화
model.enable_input_require_grads()

# LoRA 적용
model = get_peft_model(model, lora_config)

# gradient가 필요한 파라미터 확인
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"학습 가능: {name}")
        break  # 첫 번째만 확인

# 학습 가능한 파라미터 확인
model.print_trainable_parameters()

모델 모듈 이름 확인:
  transformer.h.0.attn
  transformer.h.0.attn.c_attn
  transformer.h.0.attn.c_proj
  transformer.h.0.attn.attn_dropout
  transformer.h.0.attn.resid_dropout
  transformer.h.0.mlp.c_proj
  transformer.h.1.attn
  transformer.h.1.attn.c_attn
  transformer.h.1.attn.c_proj
  transformer.h.1.attn.attn_dropout
  transformer.h.1.attn.resid_dropout
  transformer.h.1.mlp.c_proj
  transformer.h.2.attn
  transformer.h.2.attn.c_attn
  transformer.h.2.attn.c_proj
  transformer.h.2.attn.attn_dropout
  transformer.h.2.attn.resid_dropout
  transformer.h.2.mlp.c_proj
  transformer.h.3.attn
  transformer.h.3.attn.c_attn
  transformer.h.3.attn.c_proj
  transformer.h.3.attn.attn_dropout
  transformer.h.3.attn.resid_dropout
  transformer.h.3.mlp.c_proj
  transformer.h.4.attn
  transformer.h.4.attn.c_attn
  transformer.h.4.attn.c_proj
  transformer.h.4.attn.attn_dropout
  transformer.h.4.attn.resid_dropout
  transformer.h.4.mlp.c_proj
  transformer.h.5.attn
  transformer.h.5.attn.c_attn
  transfo

## 5. 데이터 로드 및 전처리

In [9]:
def load_jsonl(file_path):
    """JSONL 파일 로드"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# 데이터 로드
train_data = load_jsonl(DATA_PATH / 'train.jsonl')
val_data = load_jsonl(DATA_PATH / 'val.jsonl')

print(f"학습 데이터: {len(train_data):,}개")
print(f"검증 데이터: {len(val_data):,}개")

학습 데이터: 45,000개
검증 데이터: 5,000개


In [10]:
# 프롬프트 템플릿
PROMPT_TEMPLATE = """### 질문:
{instruction}

### 답변:
{output}</s>"""

def format_prompt(example):
    """프롬프트 형식으로 변환"""
    return PROMPT_TEMPLATE.format(
        instruction=example['instruction'],
        output=example['output']
    )

# 예시 확인
print("=" * 50)
print("프롬프트 형식 예시:")
print("=" * 50)
print(format_prompt(train_data[0]))

프롬프트 형식 예시:
### 질문:
다음은 미디어 분야의 추상물 관련 용어입니다. '차점작'의 정의를 설명해주세요.

### 답변:
최고점이나 기준점에 다음가는 점수를 받은 작품.</s>


In [11]:
# 토큰화 함수
MAX_LENGTH = 512

def tokenize_function(examples):
    """텍스트를 토큰화"""
    texts = [format_prompt(ex) for ex in examples]

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        return_tensors='pt'
    )

    # labels = input_ids (Causal LM)
    tokenized['labels'] = tokenized['input_ids'].clone()

    return tokenized

# Dataset 생성
train_texts = [format_prompt(ex) for ex in train_data]
val_texts = [format_prompt(ex) for ex in val_data]

train_dataset = Dataset.from_dict({'text': train_texts})
val_dataset = Dataset.from_dict({'text': val_texts})

print(f"Train Dataset: {len(train_dataset)}")
print(f"Validation Dataset: {len(val_dataset)}")

Train Dataset: 45000
Validation Dataset: 5000


In [12]:
# 토큰화 적용
def tokenize_dataset(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )

train_tokenized = train_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=['text']
)

val_tokenized = val_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=['text']
)

print("토큰화 완료!")
print(f"Train 샘플 키: {train_tokenized[0].keys()}")

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

토큰화 완료!
Train 샘플 키: dict_keys(['input_ids', 'attention_mask'])


## 6. 학습 설정

In [13]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=str(OUTPUT_PATH / 'checkpoints'),

    # 학습 하이퍼파라미터
    num_train_epochs=3,
    per_device_train_batch_size=4,      # Colab T4 기준
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,       # 실효 배치 사이즈 = 4 * 4 = 16

    # 학습률 설정
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,

    # 메모리 최적화
    fp16=True,
    gradient_checkpointing=True,

    # 로깅 및 저장
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=500,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,

    # 기타
    report_to='none',  # wandb 사용시 'wandb'로 변경
    seed=42,
)

print("학습 설정 완료!")
print(f"  - Epochs: {training_args.num_train_epochs}")
print(f"  - Batch Size: {training_args.per_device_train_batch_size}")
print(f"  - Gradient Accumulation: {training_args.gradient_accumulation_steps}")
print(f"  - Effective Batch Size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning Rate: {training_args.learning_rate}")

학습 설정 완료!
  - Epochs: 3
  - Batch Size: 4
  - Gradient Accumulation: 4
  - Effective Batch Size: 16
  - Learning Rate: 0.0002


In [14]:
# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM이므로 False
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

print("Trainer 생성 완료!")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainer 생성 완료!


## 7. 모델 학습

In [16]:
# 학습 시작!
print("=" * 50)
print("학습 시작!")
print("=" * 50)

trainer.train()

학습 시작!


Step,Training Loss,Validation Loss
500,1.748400,1.598804
1000,1.619100,1.510660
1500,1.593600,1.470789
2000,1.516000,1.452032
2500,1.515200,1.435205
3000,1.515500,1.423355
3500,1.488200,1.415249
4000,1.480700,1.409490
4500,1.485300,1.399825
5000,1.453400,1.395824


Step,Training Loss,Validation Loss
500,1.748400,1.598804
1000,1.619100,1.510660
1500,1.593600,1.470789
2000,1.516000,1.452032
2500,1.515200,1.435205
3000,1.515500,1.423355
3500,1.488200,1.415249
4000,1.480700,1.409490
4500,1.485300,1.399825
5000,1.453400,1.395824


TrainOutput(global_step=8439, training_loss=1.5241271063168051, metrics={'train_runtime': 9366.361, 'train_samples_per_second': 14.413, 'train_steps_per_second': 0.901, 'total_flos': 3.566941614637056e+16, 'train_loss': 1.5241271063168051, 'epoch': 3.0})

In [17]:
# 학습 결과 확인
print("\n" + "=" * 50)
print("학습 완료!")
print("=" * 50)

# 최종 평가
eval_results = trainer.evaluate()
print(f"\n검증 결과:")
for key, value in eval_results.items():
    print(f"  - {key}: {value:.4f}")


학습 완료!



검증 결과:
  - eval_loss: 1.3834
  - eval_runtime: 97.9574
  - eval_samples_per_second: 51.0430
  - eval_steps_per_second: 12.7610
  - epoch: 3.0000


## 8. 모델 저장

In [18]:
# LoRA 어댑터 저장
model.save_pretrained(OUTPUT_PATH / 'lora_adapter')
tokenizer.save_pretrained(OUTPUT_PATH / 'lora_adapter')

print(f"모델 저장 완료!")
print(f"저장 위치: {OUTPUT_PATH / 'lora_adapter'}")

모델 저장 완료!
저장 위치: /content/drive/MyDrive/LLM_FineTuning_Project/model/lora_adapter


In [19]:
# (선택) LoRA를 베이스 모델과 병합하여 전체 모델 저장
merged_model = model.merge_and_unload()
merged_model.save_pretrained(OUTPUT_PATH / 'merged_model')
tokenizer.save_pretrained(OUTPUT_PATH / 'merged_model')

print(f"병합 모델 저장 완료!")
print(f"저장 위치: {OUTPUT_PATH / 'merged_model'}")

병합 모델 저장 완료!
저장 위치: /content/drive/MyDrive/LLM_FineTuning_Project/model/merged_model
